<a href="https://colab.research.google.com/github/dvschultz/SRFBN_CVPR19/blob/master/SRFBN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dvschultz/SRFBN_CVPR19
%cd SRFBN_CVPR19/models
!gdown --id 1Dsb_-OH0CeSJVjvP9A4bh2_IBQh9R-ja
!unzip SRFBN_CVPR19_Models.zip
#!gdown --id 1NYIS0zPrpSO5fv1xpN1hH6qX4lBYxIW5
#!unzip SRFBN-S_CVPR19_Models.zip
#!rm /content/SRFBN_CVPR19/models/SRFBN-S_CVPR19_Models.zip
!rm /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models.zip
%cd ../
!mv /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models/*.pth /content/SRFBN_CVPR19/models/
!rmdir /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models
rm /content/SRFBN_CVPR19/results/LR/MyImage/*.png

Cloning into 'SRFBN_CVPR19'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 293 (delta 33), reused 12 (delta 4), pack-reused 212
Receiving objects: 100% (293/293), 3.53 MiB | 33.80 MiB/s, done.
Resolving deltas: 100% (103/103), done.
/content/SRFBN_CVPR19/models
Downloading...
From: https://drive.google.com/uc?id=1Dsb_-OH0CeSJVjvP9A4bh2_IBQh9R-ja
To: /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models.zip
52.9MB [00:00, 145MB/s] 
Archive:  SRFBN_CVPR19_Models.zip
   creating: SRFBN_CVPR19_Models/
  inflating: SRFBN_CVPR19_Models/SRFBN_x2_BI.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x3_DN.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x3_BD.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x3_BI.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x4_BI.pth  
/content/SRFBN_CVPR19


In [3]:
!python test.py -opt options/test/test_SRFBN_LR_only_x2.json

===> Export CUDA_VISIBLE_DEVICES = [0]
===> [LR] Dataset is created.
===> Test Dataset: [MyImage]   Number of images: [27]
===> Building network [SRFBN]...
===> Loading model from [./models/SRFBN_x2_BI.pth]...
===> Network Summary

SRFBN(
  (sub_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (conv_in): Sequential(
    (0): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): PReLU(num_parameters=1)
  )
  (feat_in): Sequential(
    (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): PReLU(num_parameters=1)
  )
  (block): FeedbackBlock(
    (compress_in): Sequential(
      (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): PReLU(num_parameters=1)
    )
    (upBlocks): ModuleList(
      (0): Sequential(
        (0): ConvTranspose2d(64, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (1): PReLU(num_parameters=1)
      )
      (1): Sequential(
        (0): ConvTranspose2d(64, 64, kernel_size=(6, 6), stride=(2, 2)

In [4]:
!zip -r saved-bbvday2020.zip /content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2

  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/ (stored 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed5587.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed51504.png (deflated 1%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed0263.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed1388.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed0219-2.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed0023.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed51586.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed1029_x_seed1024.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed0219.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRFBN/x2/seed0266.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/SR/MyImage/SRF